# ![LOGO](utlogo.png)

### تمرین سوم درس الگوریتم‌های یادگیری ماشین - امیرحسین مهدی‌نژاد - شماره دانشجویی ۸۱۰۸۰۰۰۵۸

In [9]:
from IPython.core.display import HTML
HTML("<style>.jp-RenderedMarkdown{direction:rtl;display:flex;flex-direction:column;font-family:'IRANSans,Tahoma,Helvetica,sans-serif';margin-top:10px;font-size:1.1rem;direction:rtl}mark{color:#FFF;padding:3px; background-color:#AAAAAAA0}</style>")

### سوال اول
#### اضافه کردن کتابخانه‌های مورد استفاده

In [2]:
!pip install h5py

In [1]:
import h5py
import numpy as np

#### تابع لود کردن داده‌ها

In [2]:
def load_dataset(train_path, test_path):
    train_dataset = h5py.File(train_path, "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File(test_path, "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

#### پیاده‌سازی تابع زیگموید
##### یک مقدار را به عنوان پارامتر ورودی گرفته و sigmoid آن را محاسبه می‌کند
$$\sigma(z) = \frac{1}{1+e^{-z}}$$

In [ ]:
def sigmoid(z):
    '''
        input: z (a scalar)
        output: sigmoid(z)
    '''
    return 1/(1 + np.exp(-z))

#### مقدار دهی weights و bias
##### وکتور w و b با صفر مقداردهی اولیه می‌شوند

In [ ]:
def initialize_with_zeros(dim):
    '''
        input: dim (size of the w vector)
        output: - w: initilized vector of size (dim, 1)
                - b: initialized scalar
    '''
    return np.zeros((dim, 1)), 0

#### پیاده‌سازی forward propagation و backward propagation برای یادگیری پارامترها
#####  بدین منظور از forward propagation برای محاسبه‌ی تابع هزینه و خروجی y استفاده شده و همچنین برای محاسبه‌ی گرادیان نزولی backward propagation را داریم.

$$ a^{(i)} = \sigma(w^Tx^{(i)} + b) $$
$$ J = \frac{1}{m}\sum_{i=1}^{m}-y^{(i)}\log(a^{(i)})-(1-y^{(i)})\log(1-a^{(i)}) $$
$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T $$
$$ \frac{\partial J}{\partial b} = \frac{1}{m}\sum_{i=1}^{m}(a^{(i)}-y^{(i)}) $$

In [ ]:
def propagate(w, b, x, y):
    '''
        input: - w: weights, a numpy array of size (px * px * 3, 1)
               - b: bias
               - x: data of size (px * px * 3, number of examples)
               - y: label vector of size (1, number of examples)  

        output:
               - cost: cost for logistic regression
               - dw: gradient of the loss with respect to w
               - db: gradient of the loss with respect to b
    '''
    m = x.shape[1]
    
    # Forward Propagation:
    activation = sigmoid(np.dot(w.T, x) + b)
    cost = -(1/m)*(np.sum(y*np.log(activation) + (1-y)*np.log(1-activation)))

    # Backward Propagation
    gradient = {'dw': (1/m)*np.dot(x, (activation-y).T), 
                'db': (1/m)*np.sum(activation-y)}

    return gradient, cost

#### کمینه کردن مقدار تابع هزینه
##### در واقع با بکارگیری الگوریتم گرادیان کاهشی پارامترهای b و w بهینه می‌شوند. هزینه در هر ۱۰۰ اجرا یک بار چاپ می‌شود

In [ ]:
def optimize(w, b, x, y, num_iterations, lr):
    '''
        input: - w: weights, a numpy array of size (px * px * 3, 1)
               - b: bias
               - x: data of size (px * px * 3, number of examples)
               - y: label vector of size (1, number of examples) 
               - num_iterations: number of iterations of the optimization loop
               - lr: learning_rate, for the GD update rule
        
        output:
               - params: dictionary containing w and b
               - grads: dictionary containing the gradients of the weights and bias with respect to the cost function
               - costs: list of all the costs computed during the optimization, this will be used to plot the learning curve
    '''
    costs = []

    for i in range(num_iterations):
        gradient, cost = propagate(w, b, x, y)

        dw, db = gradient['dw'], gradient['db']
        
        w -= lr*dw
        b -= lr*db
        
        if i % 100 == 0:
            costs.append(cost)
            print('{}th iteration cost: {:. 2f}'.format(i, cost))

    return {'w': w, 'b': b},\
            {'dw': dw, 'db': db}\
            costs

#### بروزرسانی مقادیر با گرادیان کاهشی
##### پس از تعیین خروجی و تابع هزینه، باید مقادیر بهینه را پیدا کنیم تا ارور را به حداقل برسانیم و هزینه را کاهش دهیم.

In [ ]:
def predict(w, b, x):
    '''
        input: - w: weights (px * px * 3, 1)
               - b: bias
               - x: data of size (px * px * 3, number of examples)
        
        output:
               - y_pred: np array containing all predictions (0 / 1) for examples in x
    '''
    m = x.shape[1]
    y_pred = np.zeros((1, m))
    w = w.reshape(x.shape[0], 1)
    activation = sigmoid(np.dot(w.T, x) + b)

    for i in range(activation.shape[1]):
        y_pred[0,i] = 1 if activation[0, i] > 0.5 else 0

    return y_pred

##### هدف رگرسیون لجستیکی کاهش وزن‌ها و بهبود صحت خروجی است که با استفاده از تابع گرادیان کاهشی بدست می‌آید.
##### طبق خواسته‌ی تمرین، در تابع model از توابع تعریف شده برای پیاده‌سازی رگرسیون لجستیکی استفاده می‌کنیم

In [ ]:
def model(X_train, y_train, X_test, y_test, num_iterations = 2000, lr = 0.5):
   
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, gradients, costs = optimize(w, b, X_train, y_train, num_iterations, lr)
    
    w, b = parameters['w'], parameters['b']

    y_pred_train = predict(w, b, X_train)
    y_pred_test = predict(w, b, X_test)
    
    print('train acc: {:. 2f}'.format(100 - np.mean(np.abs(y_pred_train - y_train)) * 100),\
        'test acc: {:. 2f}'.format(100 - np.mean(np.abs(y_pred_test - y_test)) * 100) )

    return {'y_pred_train': y_pred_train,
            'y_pred_test': y_pred_test,
            'w': w,
            'b': b,
            'costs': costs}

##### اجرای قطعه کد برای داده‌های تمرین